# Prediction of tumor immune cell infiltration based on extracellular matrix organization

In [117]:
# Import modules
import auxiliary
import gzip
import matplotlib.pyplot as plt
import numpy as np


## Lecture du jeu de données

On s'intéresse ici à des descripteurs qui sont extraits à partir de différentes positions au sein d'images de coupe histologique de tumeurs.

In [47]:
filepath_wt = "../data/WTconcatenate.csv.gz"
filepath_ki = "../data/KIconcatenate.csv.gz"

df_wt = auxiliary.read_dataframe(filepath_wt, low_memory=False)
df_ki = auxiliary.read_dataframe(filepath_ki, low_memory=False)

### Conversion du type des colonnes

On attribut à chaque colonne un type spécifique, pour les entier nous les convertissons en type int32 ou uint32 pour les variables strictements positives (ici, X et Y).

Les variables à valeurs flottantes sont convertis en float64. Et les variables informant sur la condition (WT/CD64-hDTR), le fichier associé ainsi que le type cellulaire étudié (Ly6/CD3) sont convertis en np.object.

Cela est pratique afin de clarifier les types associés aux variables, et pour la gestion de l'espace mémoire.

In [133]:
# Assigned columns to types
str_columns = ["Condition", "FileName", "Type"]
integer_columns = ["Mask"]
nonsigned_columns = ["X", "Y"]
float_columns = [
    "Coherency100", "Energy100", "MeanInt100", "VarInt100", "Density100", "VarDensity100",
    "Coherency140", "Energy140", "MeanInt140", "VarInt140", "Density140", "VarDensity140",
    "Coherency20", "Energy20", "MeanInt20", "VarInt20", "Density20", "VarDensity20",
    "Coherency60", "Energy60", "MeanInt60", "VarInt60", "Density60", "VarDensity60",
    "OrientationRef20", "OrientationRef60", "OrientationRef100", "OrientationRef140",
    "Angle20", "Angle60", "Angle100", "Angle140",
    "Dist", "MinDist", "MedDist", "CellArea", "CellEcc",
    "Cells100um", "MinDist100um", "MedDist100um", "CellArea100um", "CellEcc100um",
    "Frac", "Cells"
]

# Associate a type to each columns
data_type = {
    **dict.fromkeys(str_columns, object),
    **dict.fromkeys(nonsigned_columns, np.uint32),
    **dict.fromkeys(float_columns, np.float64),
    **dict.fromkeys(integer_columns, np.int32)
}

df_wt = df_wt.astype(data_type)
df_ki = df_ki.astype(data_type)

# Fusion des deux jeu de données
df_all = pd.concat([df_wt, df_ki])

## Lymphocytes T

Dans un premier temps, nous nous intéressons seulement au lymphocytes T dans l'étude de la corrélation des descripteurs (forme de la matrice, distribution) avec leur présence.

In [153]:
# Keep only T cells
df_all_cd3 = df_all[df_all["Type"] == "CD3"]

# Print
print("Lymphocytes T")
print(f"Nombre de lignes au sein du jeu de données : {df_all_cd3.shape[0]}")
print(f"Nombre de colonnes au sein du jeu de données : {df_all_cd3.shape[1]}")